In [56]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import operator
import os
from PIL import Image
from tqdm import tqdm_notebook as tqdm

from shutil import copyfile

# keras : librairie de deep learning
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
from keras.callbacks import ReduceLROnPlateau

import os
import sys
import shutil

## Creation des fichiers train et test

In [57]:
os.chdir("/home/sid2019-16/Bureau/interpromo_2020/data_Interpromo2020/Interpromo2020/script/")

In [58]:
def split_train_test_Hack(new_paths, path, airbus_planes = ['A320', 'A321', 'A350', 'A330'], split_limit = .7, s = 8):
    for plane in airbus_planes:

        shutil.rmtree(new_paths[0] + '/' + plane, ignore_errors=True)
        os.makedirs(new_paths[0] + '/' + plane)
        shutil.rmtree(new_paths[1] + '/' + plane, ignore_errors=True)
        os.makedirs(new_paths[1] + '/' + plane)
        
        picts = os.listdir(path + '/' + plane)
        picts = [pic for pic in picts if pic[-4:]=='.jpg']
        random.seed(a = s)
        random.shuffle(picts)
        
        for pict in picts[:int(split_limit*len(picts))]:
            copyfile(path + plane + '/' + pict, new_paths[0] + '/' + plane + '/' + pict)
        for pict in picts[int(split_limit*len(picts)):]:
            copyfile(path + plane + '/' + pict, new_paths[1] + '/' + plane + '/' + pict)

In [59]:
df = pd.read_csv('./../G7_images_sociales_2020-master/CSV_annotate/SEATGURU/g7_SEATGURU_annotate.csv', sep = ';')

In [60]:
for i in df.index :
    if df['view'][i] != 'Int' or df['aircraft_manufacturer'][i] != 'Airbus' :
        df.drop([i], inplace = True)

In [61]:
liste_modele = ['A320', 'A321', 'A330', 'A350']
for i in df.index :
    if df['aircraft_type'][i] not in liste_modele :
        df['aircraft_type'][i] = 'other'

/home/sid2019-16/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


39

In [39]:
list_A320 = []
list_A321 = []
list_A330 = []
list_A350 = []
list_other = []

for i in df.index:
    if df['aircraft_type'][i] == 'A320':
        list_A320.append(df['name'][i])
    if df['aircraft_type'][i] == 'A321':
        list_A321.append(df['name'][i])
    if df['aircraft_type'][i] == 'A330':
        list_A330.append(df['name'][i])
    if df['aircraft_type'][i] == 'A350':
        list_A350.append(df['name'][i])
    else :
        list_other.append(df['name'][i])

In [40]:
path = './../All Data/ANALYSE IMAGE/IMG SEATGURU'
new_path_train = './../G7_SEATGURU_cat/Int/Airbus/data_train'
new_path_val = './../G7_SEATGURU_cat/Int/Airbus/data_val'
new_paths = [new_path_train, new_path_val]

In [64]:
def split_train_test_Hack(new_paths, path, airbus_planes = [list_A320, list_A321, list_A330, list_A350], split_limit = .7, s = 8):
    for plane in airbus_planes:
        
        if plane == list_A320:
            mod = 'A320'
        if plane == list_A321:
            mod = 'A321'
        if plane == list_A330:
            mod = 'A330'
        if plane == list_A350:
            mod = 'A350'
        
        shutil.rmtree(new_paths[0] + '/' + mod + '/', ignore_errors=True)
        os.makedirs(new_paths[0] + '/' + mod + '/')
        shutil.rmtree(new_paths[1] + '/' + mod + '/', ignore_errors=True)
        os.makedirs(new_paths[1] + '/' + mod + '/')
        
        random.seed(a = s)
        random.shuffle(plane)
        
        for pict in plane[:int(split_limit*len(plane))]:
            copyfile(path + '/' + pict, new_paths[0] + '/' + mod + '/' + pict)
        for pict in plane[int(split_limit*len(plane)):]:
            copyfile(path + '/' + pict, new_paths[1] + '/' + mod + '/' + pict)

In [65]:
split_train_test_Hack(new_paths, path)

## Creation des jeux de train et test 

In [43]:
from keras.applications.vgg16 import preprocess_input

In [44]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

In [45]:
train_generator = train_datagen.flow_from_directory(
        './../G7_SEATGURU_cat/Int/Airbus/data_train',
        target_size=(224, 224),
        color_mode = 'rgb',
        batch_size = 32,
        class_mode='categorical',
        shuffle = True)

validation_generator = train_datagen.flow_from_directory(
        './../G7_SEATGURU_cat/Int/Airbus/data_val',
        target_size=(224, 224),
        shuffle = True,
        color_mode = 'rgb',
        batch_size = 32,
        class_mode='categorical')


Found 406 images belonging to 4 classes.
Found 176 images belonging to 4 classes.


## Application

In [46]:
# base model VVG16
base_model = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(224, 224, 3), pooling=None, classes=1000)

In [52]:
# Layers

x = base_model.output

# x = Conv2D(256, kernel_size=(3, 3))(x)
# x = keras.layers.BatchNormalization()(x)
# x = ReLU()(x)

# x = Conv2D(256, kernel_size=(3, 3))(x)
# x = keras.layers.BatchNormalization()(x)
# x = ReLU()(x)

# x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)

x = Dense(1024)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(512)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

# x = Dense(262)(x)
# x = keras.layers.BatchNormalization()(x)
# x = ReLU()(x)

# dernière couche que sert a prédire la bonne classe
predictions = Dense(4, activation = 'softmax')(x)

# this is the model we will train
model = Model(inputs = base_model.input, outputs = predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [50]:
# Define learning rate reducer
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=2, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

In [26]:
# vvg16
model.fit_generator(
        train_generator,
        epochs=5,
        validation_data = validation_generator,
        callbacks=[reduce_lr])

Epoch 1/5
13/13 [==============================] - 90s 7s/step - loss: 1.7781 - accuracy: 0.3522 - val_loss: 4.5258 - val_accuracy: 0.4489
Epoch 2/5
13/13 [==============================] - 94s 7s/step - loss: 0.4657 - accuracy: 0.8522 - val_loss: 3.4543 - val_accuracy: 0.3295
Epoch 3/5
13/13 [==============================] - 91s 7s/step - loss: 0.1306 - accuracy: 0.9704 - val_loss: 1.8013 - val_accuracy: 0.4091
Epoch 4/5
13/13 [==============================] - 99s 8s/step - loss: 0.0490 - accuracy: 0.9877 - val_loss: 1.9108 - val_accuracy: 0.4205
Epoch 5/5
13/13 [==============================] - 100s 8s/step - loss: 0.0277 - accuracy: 0.9951 - val_loss: 2.2032 - val_accuracy: 0.4659


In [53]:
# vvg16
model.fit_generator(
        train_generator,
        epochs=5,
        validation_data = validation_generator,
        callbacks=[reduce_lr])

Epoch 1/5
13/13 [==============================] - 89s 7s/step - loss: 1.9051 - accuracy: 0.4212 - val_loss: 6.1103 - val_accuracy: 0.4545
Epoch 2/5
13/13 [==============================] - 102s 8s/step - loss: 0.2520 - accuracy: 0.9286 - val_loss: 1.9473 - val_accuracy: 0.4716
Epoch 3/5
13/13 [==============================] - 94s 7s/step - loss: 0.0805 - accuracy: 0.9852 - val_loss: 3.3580 - val_accuracy: 0.5170
Epoch 4/5
13/13 [==============================] - 93s 7s/step - loss: 0.0380 - accuracy: 0.9926 - val_loss: 1.3094 - val_accuracy: 0.5170
Epoch 5/5
13/13 [==============================] - 91s 7s/step - loss: 0.0312 - accuracy: 0.9901 - val_loss: 2.2495 - val_accuracy: 0.5170

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.


In [54]:
model.save('model_int_airbus.h5')